In [ ]:
%pip install pymongo tqdm sentence-transformers

Python interpreter will be restarted.
  Using cached pymongo-4.3.3-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (492 kB)
  Using cached tqdm-4.65.0-py3-none-any.whl (77 kB)
  Using cached dnspython-2.3.0-py3-none-any.whl (283 kB)
Python interpreter will be restarted.


In [ ]:
import pandas as pd
import json
from collections import Counter
from tqdm import tqdm
from pymongo import MongoClient
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.sql import Window
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
conf = pyspark.SparkConf()


import copy
import numpy as np

from sentence_transformers import SentenceTransformer

# Load the model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')


tqdm.pandas()

MONGO_CONN = 'mongodb+srv://<username>:<password>@retail-demo.2wqno.mongodb.net/?retryWrites=true&w=majority'

In [ ]:
atp = spark.read.format("mongodb").\
            option('spark.mongodb.connection.uri', MONGO_CONN).\
            option('spark.mongodb.database', "search").\
            option('spark.mongodb.collection', "atp_status_myn").\
            option("spark.mongodb.partitioner","com.mongodb.spark.sql.connector.read.partitioner.SamplePartitioner").\
            option("spark.mongodb.read.partitioner.options.partition.field", "_id").\
            option("spark.mongodb.read.partitioner.options.partition.size",1).\
            option('spark.mongodb.aggregation.pipeline',[]).\
            option("forceDeleteTempCheckpointLocation", "true").load()

atp = atp.drop("_id")
atp = atp.withColumn("id", F.col("id").cast("string"))

In [ ]:
images=spark.read.format("mongodb").\
            option('spark.mongodb.connection.uri', MONGO_CONN).\
            option('spark.mongodb.database', "search").\
            option('spark.mongodb.collection', "mynimages").\
            option("spark.mongodb.partitioner","com.mongodb.spark.sql.connector.read.partitioner.SamplePartitioner").\
            option("spark.mongodb.read.partitioner.options.partition.field", "_id").\
            option("spark.mongodb.read.partitioner.options.partition.size",1).\
            option('spark.mongodb.aggregation.pipeline',[{"$project":{'product_id':0}}]).\
            option("forceDeleteTempCheckpointLocation", "true").load()

images = images.drop("_id")
images = images.drop("filename")
images = images.withColumn("id", F.col("id").cast("string"))

In [ ]:
prd_desc=spark.read.format("mongodb").\
            option('spark.mongodb.connection.uri', MONGO_CONN).\
            option('spark.mongodb.database', "search").\
            option('spark.mongodb.collection', "prd_desc_myn").\
            option("spark.mongodb.partitioner","com.mongodb.spark.sql.connector.read.partitioner.SamplePartitioner").\
            option("spark.mongodb.read.partitioner.options.partition.field", "_id").\
            option("spark.mongodb.read.partitioner.options.partition.size",1).\
            option('spark.mongodb.aggregation.pipeline',[]).\
            option("forceDeleteTempCheckpointLocation", "true").load()

prd_desc = prd_desc.drop("_id")
prd_desc = prd_desc.withColumn("title", F.col("productDisplayName"))
prd_desc = prd_desc.withColumn("mfg_brand_name", F.col("brandName"))

prd_desc = prd_desc.withColumn("id", F.col("id").cast(T.StringType()))

In [ ]:
prd_score=spark.read.format("mongodb").\
            option('spark.mongodb.connection.uri', MONGO_CONN).\
            option('spark.mongodb.database', "search").\
            option('spark.mongodb.collection', "prd_score_myn").\
            option("spark.mongodb.partitioner","com.mongodb.spark.sql.connector.read.partitioner.SamplePartitioner").\
            option("spark.mongodb.read.partitioner.options.partition.field", "_id").\
            option("spark.mongodb.read.partitioner.options.partition.size",1).\
            option('spark.mongodb.aggregation.pipeline',[]).\
            option("forceDeleteTempCheckpointLocation", "true").load()

prd_score = prd_score.drop("_id")
prd_score = prd_score.withColumn("id", F.col("id").cast(T.StringType()))

In [ ]:
price = spark.read.format("mongodb").\
            option('spark.mongodb.connection.uri', MONGO_CONN).\
            option('spark.mongodb.database', "search").\
            option('spark.mongodb.collection', "price_myn").\
            option("spark.mongodb.partitioner","com.mongodb.spark.sql.connector.read.partitioner.SamplePartitioner").\
            option("spark.mongodb.read.partitioner.options.partition.field", "_id").\
            option("spark.mongodb.read.partitioner.options.partition.size",1).\
            option('spark.mongodb.aggregation.pipeline',[{"$project":{"_id":0}}]).\
            option("forceDeleteTempCheckpointLocation", "true").load()

price = price.drop("_id")
price = price.withColumn("id", F.col("id").cast(T.StringType()))

In [ ]:
final = atp.join(F.broadcast(images), on='id',how='inner').\
    join(F.broadcast(prd_desc),on='id',how='inner').\
    join(F.broadcast(prd_score),on='id',how='inner').\
    join(F.broadcast(price),on='id',how='inner')


@F.udf()
def get_vec(text):
    return model.encode(text)

final.withColumn("vec", get_vec("title"))

final.write.format("mongodb").\
            option('spark.mongodb.connection.uri', MONGO_CONN).\
            option('spark.mongodb.database', "search").\
            option('spark.mongodb.collection', "catalog_myn").\
            option('spark.mongodb.idFieldList', 'id').\
            mode("append").\
            save()

In [ ]:
final.count()

Out[8]: 339092

In [ ]:
images.count()

Out[9]: 44446

In [ ]:
atp.count()

Out[10]: 44446

In [ ]:
price.count()

Out[11]: 44446

In [ ]:
prd_desc.count()


Out[12]: 44446